In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")  
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [3]:
url = "https://finance.yahoo.com/"
driver.get(url)
time.sleep(5)  

In [4]:
tabs = ["US", "EUROPE", "ASIA", "RATES", "COMMODITIES", "CURRENCIES", "CRYPTOCURRENCIES"]
data = []

In [5]:
for tab in tabs:
    try:
        btn = driver.find_element("id", f"tab-{tab}")
        driver.execute_script("arguments[0].click();", btn)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        tickers = soup.select("li.box-item")

        for item in tickers:
            name = item.select_one("span.symbol").text.strip()
            price = item.select_one("fin-streamer[data-field='regularMarketPrice']").text.strip()
            change = item.select_one("fin-streamer[data-field='regularMarketChange']").text.strip()
            percent = item.select_one("fin-streamer[data-field='regularMarketChangePercent']").text.strip()

            data.append({
                "Tab": tab,
                "Name": name,
                "Price": price,
                "Change": change,
                "Percent Change": percent
            })

    except Exception as e:
        print(f"❌ فشل تحميل تبويب {tab}: {e}")

driver.quit()

In [6]:
df = pd.DataFrame(data)
df


,Tab,Name,Price,Change,Percent Change
0,US,S&P Futures,"5,905.75",-69.75,(-1.17%)
1,US,Dow Futures,"42,405.00",-331.00,(-0.77%)
2,US,Nasdaq Futures,"21,166.50",-339.50,(-1.58%)
3,US,Russell 2000 Futures,"2,086.40",-33.00,(-1.56%)
4,US,VIX,19.71,+2.47,(+14.31%)
5,US,Gold,"3,242.00",+54.80,(+1.72%)
6,EUROPE,FTSE 100,"8,624.49",-60.07,(-0.69%)
7,EUROPE,CAC 40,"7,830.65",-56.04,(-0.71%)
8,EUROPE,DAX P,"23,710.04",-57.39,(-0.24%)
9,EUROPE,Euronext 100 Index,"1,582.96",-13.98,(-0.88%)


In [7]:
import pyodbc,  sys 

conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'

                      'SERVER=.\\SQLEXPRESS;'

                      'DATABASE=yahoo finance;'

                      'Trusted_Connection=yes;'
                      
                      'Encrypt=no')
cursor = conn.cursor()

In [8]:
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO MarketSummary (Tab, Name, Price, Change, PercentChange)
        VALUES (?, ?, ?, ?, ?)
    """, row["Tab"], row["Name"], row["Price"], row["Change"], row["Percent Change"])

conn.commit()